In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline
import random
import math

In [ ]:
import torchvision
import torchvision.transforms as transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

Device available now: cuda


In [ ]:
image_transforms = transforms.Compose(
         [transforms.ToTensor(),
         transforms.CenterCrop(28),
         transforms.Normalize((0.5), (0.5))]
)

In [ ]:
batchsize = 32
rowsize = 28
colsize = 28
inchannels = 3
fc_size = 800
no_classes = 10
epochs = 900


In [ ]:
train_zero = []
train_abs = []
train_exp = []
test_zero = 0
test_abs = 0
test_exp = 0

In [ ]:
train = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=image_transforms)
test = torchvision.datasets.CIFAR10(root='./', train=False, transform=image_transforms)

Files already downloaded and verified


In [ ]:
traindata = torch.utils.data.DataLoader(train,batch_size=batchsize, shuffle=True, num_workers=2,pin_memory=True)
testdata = torch.utils.data.DataLoader(test,batch_size=batchsize, shuffle=False, num_workers=2,pin_memory=True)

In [ ]:
class IOC_MLP(nn.Module):
  def __init__(self,rowsize,colsize,fc_size,inchannels,no_classes):
    super(IOC_MLP, self).__init__()
    self.fc1 = nn.Linear(rowsize*colsize*inchannels,fc_size)
    self.batchnorm1 = nn.BatchNorm1d(fc_size)
    self.fc2 = nn.Linear(fc_size,fc_size)
    self.batchnorm2 = nn.BatchNorm1d(fc_size)
    self.fc3 = nn.Linear(fc_size,fc_size)
    self.batchnorm3 = nn.BatchNorm1d(fc_size)
    self.fc4 = nn.Linear(fc_size,no_classes)


  def forward(self, x):
    x = torch.flatten(x,1)
    x = x.view(-1, rowsize*colsize*inchannels)
    x = F.elu(self.fc1(x))
    x = self.batchnorm1(x)
    x = F.elu(self.fc2(x))
    x = self.batchnorm2(x)
    x = F.elu(self.fc3(x))
    x = self.batchnorm3(x)
    return x

In [ ]:
model = IOC_MLP(rowsize,colsize,fc_size,inchannels,no_classes).to(device)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr= 0.0001)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
# def training(model,traindata,testdata)
# print("Choose the weight Variation number, (1 for clipping negative weight to zero , 2 for taking absolute of weights , 3 for exponentiation of weights")
# a = input()
for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  for i,data in enumerate(traindata):
    # iterating through every batch
    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
    for w in model.parameters():
      # F.relu(w)
      torch.where(w > 0, w, 0)
  train_zero.append(num_correct/tot)
  print("Train Accuracy for clipping negative weights to zero: ",num_correct/tot)
  print("Loss for clipping negative weights to zero : ",loss.item())
  

    

In [4]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testdata:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    test_zero = correct/total
    print("Test Accuracy for clipping negative weights to zero : ",100 * correct / total)


Test Accuracy for clipping negative weights to zero : 60.50312622138613


In [ ]:
model = IOC_MLP(rowsize,colsize,fc_size,inchannels,no_classes).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr= 0.0001)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
# def training(model,traindata,testdata)
# print("Choose the weight Variation number, (1 for clipping negative weight to zero , 2 for taking absolute of weights , 3 for exponentiation of weights")
# a = input()
for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  for i,data in enumerate(traindata):
    # iterating through every batch
    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
    for w in model.parameters():
      torch.abs(w)
    # F.relu(model.parameters())
  train_abs.append(num_correct/tot)    
  print("Train Accuracy for absolute values for negative weights: ",num_correct/tot)
  print("Loss for absolute values for negative weights: ",loss.item())
  

In [5]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testdata:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    test_abs = correct/total
    print("Test Accuracy on absolute weights : ",100 * correct / total) 


Test Accuracy on absolute weights : 61.40856191143649


In [ ]:
model = IOC_MLP(rowsize,colsize,fc_size,inchannels,no_classes).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr= 0.0001)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
 # def training(model,traindata,testdata)
# print("Choose the weight Variation number, (1 for clipping negative weight to zero , 2 for taking absolute of weights , 3 for exponentiation of weights")
# a = input()
for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  for i,data in enumerate(traindata):
    # iterating through every batch
    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
    for w in model.parameters():
      p = w
      torch.where(w > 0, w, torch.exp(p))
    # F.relu(model.parameters())
  train_exp.append(num_correct/tot)    
  print("Accuracy for exponentiation on negative weights : ",num_correct/tot)
  print("Loss for exponentiation on negative weights : ",loss.item())
  

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testdata:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    test_exp = correct/total
    print("Test Accuracy for exponentiation of weights : ",100 * correct / total)


Test Accuracy for exponentiation of weights : 65.60812452142689
